# Advanced Bonus Material
# Online scheduled evaluations
Inspiration: https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/online-evaluation?tabs=windows

## Documents


## Prerequisites
Source https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/online-evaluation?tabs=windows

### Create User Managed Identity (UMI)

https://learn.microsoft.com/en-us/entra/identity/managed-identities-azure-resources/how-manage-user-assigned-managed-identities?pivots=identity-mi-methods-azcli

```
az identity create -g \<RESOURCE GROUP\> -n \<USER ASSIGNED IDENTITY NAME\>
```

IMPORTANT: Add UMI clientId in environment variables (.env file)

### Assign UMI to Azure AI Foundry project

https://learn.microsoft.com/en-us/azure/machine-learning/how-to-identity-based-service-authentication?view=azureml-api-2&tabs=cli#add-a-user-assigned-managed-identity-to-a-workspace-in-addition-to-a-system-assigned-identity

1. Create iac/umi.yml
```
identity:
    type: system_assigned,user_assigned
    tenant_id: \<TENANT_ID\>
    user_assigned_identities:
        '/subscriptions/\<SUBSCRIPTION_ID\>/resourceGroups/\<RESOURCE_GROUP\>/providers/Microsoft.ManagedIdentity/userAssignedIdentities/<\USER_MANAGED_ID\>': {}
```

2. Update workspace:
```
cd iac
az ml workspace update --resource-group \<RESOURCE_GROUP\> --name \<WORKSPACE_NAME\> --file umi.yml
```

### Assign roles to UMI at relevant resources:

1. 'Log Analytics Contributor' role at Application Insights

2. 'Cognitive Services OpenAI Contributor' role at Azure AI Services

3. 'Storage Blob Data Contributor' role at Storage account

## Setup

### Common packages

In [ ]:
import os
import dotenv
from pathlib import Path

### Global settings

In [ ]:
# Global variables
PRIVATE = False
DATA_DIR = Path("data")
TMP_DIR = Path("tmp")

### Load environment variables

In [ ]:
# Import override environment variables from .env file
# or from private.env file if PRIVATE is True
dotenv.load_dotenv('.env' if not PRIVATE else 'private.env', override=True)

### Sanity check

In [ ]:
userManagedIdentityClientId = os.environ.get("AZURE_AI_FOUNDRY_UMI_CLIENT_ID")
if userManagedIdentityClientId is None:
    print("FATAL: No user managed identity client id provided. Using system assigned identity.")

### Azure credentials

In [ ]:
# https://learn.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

### Get Azure AI Foundry project client

In [ ]:
from azure.ai.projects import AIProjectClient

# Create an Azure AI Client from a connection string. Available on Azure AI project Overview page.
# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.aiprojectclient?view=azure-python-preview
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ.get("AZURE_AI_PROJECT_CONNECTION_STRING"),
)

### Config dictionaries used by Azure AI SDK

In [ ]:
# Configuration for Azure AI Foundry project
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP_AI"),
    "project_name": os.environ.get("AZURE_AI_FOUNDRY_PROJECT_NAME"),
}

# Configuration for Azure OpenAI model
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
    "type": "azure_openai"
}

## Get custom evaluator from Azure AI foundry library

### Connect to Azure AI Foundry project

In [ ]:
from azure.ai.ml import MLClient

# Define ml_client to register custom evaluator
# https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.mlclient?view=azure-python
ml_client = MLClient(
       subscription_id=os.environ["AZURE_SUBSCRIPTION_ID"],
       resource_group_name=os.environ["AZURE_RESOURCE_GROUP_AI"],
       workspace_name=os.environ["AZURE_AI_FOUNDRY_PROJECT_NAME"],
       credential=credential
)

### Helper to built evaluator library id

In [ ]:
from azure.ai.ml.entities import Model

def get_evaluator_library_id(_evaluator: Model) -> str:
    _ws = ml_client.workspaces.get(ml_client.workspace_name)
    _id=f"azureml://locations/{_ws.location}/workspaces/{_ws._workspace_id}/models/{_evaluator.name}/versions/{_evaluator.version}"
    print(f"{_evaluator.name} library id: {_id}")
    return _id

In [ ]:
_evaluator = ml_client.evaluators.get("FriendlinessEvaluator", label="latest")
friendlinessEvaluator_libId = get_evaluator_library_id(_evaluator)

## Configure and schedule evaluation

### Load Kusto query

In [ ]:
# https://learn.microsoft.com/en-us/python/api/azure-monitor-opentelemetry/azure.monitor.opentelemetry?view=azure-python    
# https://learn.microsoft.com/en-us/azure/ai-foundry/how-to/online-evaluation?tabs=windows

# Kusto Query Language (KQL) query to query data from Application Insights resource
# This query is compatible with data logged by the Azure AI Inferencing Tracing SDK (linked in documentation)
# You can modify it depending on your data schema
# The KQL query must output these required columns: operation_ID, operation_ParentID, and gen_ai_response_id
# You can choose which other columns to output as required by the evaluators you are using

# load string from file "traces_query.kql" into variable KUSTO_QUERY
with open("traces_query.kql", "r") as file:
    KUSTO_QUERY = file.read()

### Configure Application Insights query

In [ ]:
from azure.ai.projects.models import ApplicationInsightsConfiguration

# Connect to your Application Insights resource 
app_insights_config = ApplicationInsightsConfiguration(
    resource_id=os.environ["APP_INSIGHTS_RESOURCE_ID"],
    query=KUSTO_QUERY
)

### Configure evaluation schedule

In [ ]:
# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.recurrencetrigger?view=azure-python-preview
from azure.ai.projects.models import RecurrenceTrigger

# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.evaluationschedule?view=azure-python-preview
from azure.ai.projects.models import EvaluationSchedule

# https://learn.microsoft.com/en-us/python/api/azure-ai-projects/azure.ai.projects.models.evaluatorconfiguration?view=azure-python-preview
from azure.ai.projects.models import EvaluatorConfiguration

# Dictionary of evaluators

# https://learn.microsoft.com/en-us/python/api/azure-ai-evaluation/azure.ai.evaluation.coherenceevaluator?view=azure-python-preview
from azure.ai.evaluation import CoherenceEvaluator

evaluators = {
    "friendliness": EvaluatorConfiguration(
            id=friendlinessEvaluator_libId,
            init_params={
                "model_config": model_config
            },
            
            data_mapping={
            "response": "${data.Output}"
            } 
        ),
    "coherence" : EvaluatorConfiguration(
            id=CoherenceEvaluator.id,
            init_params={"model_config": model_config},
            data_mapping={"query": "${data.Input}", "response": "${data.Output}"}
        )
}

# Frequency to run the schedule
recurrence_trigger = RecurrenceTrigger(frequency="day", interval=1)

# Configure the online evaluation schedule
evaluation_schedule = EvaluationSchedule(
    data=app_insights_config,
    evaluators=evaluators,
    trigger=recurrence_trigger,
    description='Online scheduled evaluation',
    properties={"AzureMSIClientId": userManagedIdentityClientId }
)

### Create schedule

In [ ]:
# Name of your online evaluation schedule
schedule_name = "online_eval"

# Create the online evaluation schedule 
created_evaluation_schedule = project_client.evaluations.create_or_replace_schedule(schedule_name, evaluation_schedule)
print(f"Successfully submitted the online evaluation schedule creation request - {created_evaluation_schedule.name}, currently in {created_evaluation_schedule.provisioning_state} state.")

## Disable regular schedule

In [ ]:
project_client.evaluations.disable_schedule(schedule_name)